In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading our datasets


Classifying whether a person has diabetes or not using KNN algorithm.

In [ ]:
diabetes=pd.read_csv("../input/diabetescsv/diabetes.csv")

In [ ]:
diabetes.describe()

Looking inside diabetes datasets


In [ ]:
diabetes.head()

In [ ]:
diabetes.info()

There is no Nan or null value.

 # **Data cleaning**
**Since, some of the column have zero value such as BloodPressure,Glucose etc which has no meaning. So we are going to replace such values with mean of their respective columns.**



In [ ]:
columns=["Glucose","BloodPressure","SkinThickness","BMI","Insulin"]

In [ ]:
for i in columns:
    print('Number of zero value of ' + i,diabetes[diabetes[i]==0].shape[0])
   

In [ ]:
for i in columns:
    diabetes[i]=diabetes[i].replace(0,np.NaN)
    mean=int(diabetes[i].mean(skipna=True))
    diabetes[i]=diabetes[i].replace(np.NaN,mean)

In [ ]:
diabetes["Glucose"].mean()

In [ ]:
diabetes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(diabetes,hue="Outcome")

In [ ]:
plt.figure(figsize=(10,10))

sns.scatterplot(x="Glucose",y="BMI",hue="Outcome",data=diabetes)


In [ ]:
personwithdiabetes=diabetes[diabetes["Outcome"]==1]

In [ ]:
personwithoutdiabetes=diabetes[diabetes["Outcome"]==0]

In [ ]:
plt.figure(figsize=(6,6))
sns.kdeplot(x="Age",y="Glucose",data=diabetes,hue="Outcome")

In [ ]:
sns.kdeplot(x="Age",data=personwithdiabetes,label="Diabetes patient",shade=True)
sns.kdeplot(x="Age",data=personwithoutdiabetes,label="Non Diabetes patient",shade=True)
plt.legend()

In [ ]:
sns.kdeplot(x="Glucose",data=personwithdiabetes,label="Diabetes patient",shade=True)
sns.kdeplot(x="Glucose",data=personwithoutdiabetes,label=" non Diabetes patient",shade=True)
plt.legend()

In [ ]:
sns.kdeplot(x="BMI",data=personwithdiabetes,label="Diabetes patient",shade=True)
sns.kdeplot(x="BMI",data=personwithoutdiabetes,label=" non Diabetes patient",shade=True)
plt.legend()

# Splitting data

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x=diabetes[["Pregnancies","Glucose","BloodPressure","SkinThickness","BMI","Insulin","DiabetesPedigreeFunction","Age"]]
y=diabetes["Outcome"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
st_scal=StandardScaler()
x_train=st_scal.fit_transform(x_train)
x_test=st_scal.fit_transform(x_test)

# Using KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
Knn=KNeighborsClassifier(n_neighbors=14,p=2,metric="euclidean")

In [ ]:
Knn.fit(x_train,y_train
       )

In [ ]:
prediction=Knn.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,f1_score

In [ ]:
confusion_mat=confusion_matrix(y_test,prediction)

confusion_mat

In [ ]:
print(classification_report(y_test,prediction))

In [ ]:
print(f1_score(y_test,prediction))

In [ ]:
from sklearn.metrics import plot_confusion_matrix,accuracy_score
plt.figure(figsize=(6,6))
sns.heatmap(confusion_mat,annot=True,fmt="g")

# Finding Optimal value of k

In [ ]:
error_rate = []
for i in range(1,30):
 knn = KNeighborsClassifier(n_neighbors=i)
 knn.fit(x_train,y_train)
 predct = knn.predict(x_test)
 error_rate.append(np.mean(predct != y_test))
print(error_rate)

plt.figure(figsize=(10,6))
plt.plot(range(1,30),error_rate,color='blue', linestyle='dashed', 
         marker='o',markerfacecolor='green', markersize=8)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')
print("Minimum error:",min(error_rate),"at K =",error_rate.index(min(error_rate)))

In [ ]:
acc = []
# Will take some time
from sklearn import metrics
for i in range(1,40):
    neigh = KNeighborsClassifier(n_neighbors = i).fit(x_train,y_train)
    yhat = neigh.predict(x_test)
    acc.append(metrics.f1_score(y_test, yhat))
    
plt.figure(figsize=(10,6))
plt.plot(range(1,40),acc,color = 'blue',linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('accuracy vs. K Value')
plt.xlabel('K')
plt.ylabel('Accuracy')
print("Maximum accuracy:",max(acc),"at K =",acc.index(max(acc)))

So ,we take 14  as k value to improve accuracy of model.